In [ ]:
!pip install bs4

In [ ]:
import pandas as pd#to read csv files
import numpy as np
import matplotlib.pyplot as plt#to plot data
import seaborn as sns
import string
import re

from numpy import median

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split



from nltk.corpus import stopwords
from bs4 import BeautifulSoup

# from nltk.stem import WordNetLemmatizer
# from nltk.stem import SnowballStemmer
# from sklearn.feature_extraction.text import CountVectorizer
# from nltk import pos_tag
# from nltk.corpus import wordnet

from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
# from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
# # from keras.optimizers import RMSprop
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing import sequence
# # from keras.utils import to_categorical
# from keras.preprocessing import sequence
# from keras.models import Model

In [ ]:
#creating an object by reading csv with encoding=latin-1
dat = pd.read_csv('../input/spam-text-message-classification/SPAM text message 20170820 - Data.csv', encoding= 'latin-1')
#creating a new column which stores the len of the current message cell
dat['len'] = dat['Message'].apply(len)
#head is used to preview the first 5 lines
dat.head()

In [ ]:
#EDA applied
#it sets the figure size w=12 and height=5 and dpi=60
plt.figure(figsize=(12,5), dpi=60)
#It will declare space for subplots
plt.subplot(1,2,1)
#it will plot counts-plot-graph the number of times Category column values appear
sns.countplot(y=dat['Category'], palette="Set3")
#It will declare space for subplots
plt.subplot(1,2,2)
#it will plot bar graph with x-axix of len column and y-axix of category column
sns.barplot(x=dat['len'],y=dat['Category'],data=dat, estimator=median, palette="Set3")

In [ ]:
def preprocess(text):
    #here we will use tranlated to remove the punctuation symbols
    text = text.translate(str.maketrans('','', string.punctuation))
    #following list comprehension will only add words that are not common words like a, an, the etc
    #for this we first convert the word to lower and then check that it doesn't belong to stop-words
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    #it will return a string that joins a list with name text
    return " ".join(text)

def strip_html(text):
    #beautiful soap is used to get data from web pages by removing the html tags
    soup = BeautifulSoup(text, "html.parser")
    #returns by getting text from soup
    return soup.get_text()

def remove_between_square_brackets(text):
    #re.sub delete symbols from the text
    return re.sub('\[[^]]*\]', '', text)

def remove_between_square_brackets(text):
    #re.sub delete https protocols links from the text
    return re.sub(r'http\S+', '', text)

def denoise_text(text):
    text = preprocess(text)
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

In [ ]:
#it preprocesses the message and create new c_message column
dat['C_Message'] = dat['Message'].apply(denoise_text)
dat.head()

In [ ]:
#it replace ham with 0 and spam with 1 in category column
dat['Category'] = dat['Category'].map({'ham': 0, 'spam': 1})
dat.head()

In [ ]:
X = dat['C_Message']
Y = dat['Category']
#we used the train test split to split data to evaluate data without any biasedness
#for splitting data arrays into two subsets: for training data and for testing data. 
#With this function, you don't need to divide the dataset manually
# print(Y)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
#it transforms text into a meaningful representation of numbers which is used to fit machine algorithm
vect = TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#it is used to scale the training and test data
x_train = vect.fit_transform(x_train)
x_test = vect.transform(x_test)
print(x_train.shape)
print(x_test.shape)

In [ ]:
#now we use imported classifiers to fit the trained data to make it ready for the predict function
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier(n_neighbors=49)
mnb = MultinomialNB(alpha=0.2)
dtc = DecisionTreeClassifier(min_samples_split=7, random_state=111)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=31, random_state=111)
abc = AdaBoostClassifier(n_estimators=62, random_state=111)
bc = BaggingClassifier(n_estimators=9, random_state=111)
etc = ExtraTreesClassifier(n_estimators=9, random_state=111)
score = []
for i,j in enumerate([svc,knc,mnb,dtc,lrc,rfc,abc,bc,etc]):
    j.fit(x_train,y_train)
    pred = j.predict(x_test)
    score.append([accuracy_score(y_test,pred)])

In [ ]:
#here created a dictionary with classifier name as key and and value from the fit score list items
lab = {'SVM': [score[0]],'KNeighborClassifier': [score[1]],'MultinomialNB':[score[2]],
      'DecisionTreeClassifier':[score[3]],'LogisticRegression':[score[4]],
      'RandomForestClassifier':[score[5]],'Adaboost':[score[6]],'BaggingClassifier':[score[7]],
      'ExtraTreesClassifier':[score[8]]}

#created data frame by passing lab as a dict and column as the name of the classifier
result = pd.DataFrame(lab, columns = ['SVM','KNeighborClassifier', 'MultinomialNB',
                                  'DecisionTreeClassifier','LogisticRegression','RandomForestClassifier',
                                  'Adaboost','BaggingClassifier','ExtraTreesClassifier'])
#Took the transpose of the first five rows
np.transpose(result.head())